> 2번   
※ LogisticRegression() 클래스의 인자 max_iter는 1000 으로 지정하시오.   
Q3 에 단계1: ~~문제 5번~~ 2단계에서

In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy  as np
np.random.seed(123)

df_origin = pd.read_csv("../Datasets/edu_enrollees.csv")

삼성전자 임직원의 체계적인 커리어 패스 설계를 위해 데이터 분석과 같은 Hard Skill 역량과 Soft Skill 역량을 조화롭게 함양하기 위해 HRD 부서의 조프로는 전산팀의 협조를 받아 데이터를 확보하였다.  
  
※ 분석을 수행하기 전, 상기 데이터를 이용하여 아래의 전처리를 수행하시오.  
  
* 단계 1: 분석에 사용하지 않을 city, company_size, company_type 컬럼을 제거하시오.  
* 단계 2: 각 문자형 변수에 결측치(null/empty space)가 하나라도 존재하는 행은 모두 제거하시오.  
* 단계 3: experience 변수 값이 ‘>20’ 또는 ‘<1’인 값을 제거하고 experience 컬럼의 타입을 정수형으로 변환하시오.  
* 단계 4: last_new_job 변수의 값이 ‘>4’ 또는 ‘never’인 값을 제거하고 해당변수의 타입을 정수형으로 변환하시오.  
  
전처리 수행 이후 행 개수는 7,522이며 전처리가 완료된 데이터를 base 객체로 지정하고 이를 사용하여 문제를 풀이하시오.  
  

In [131]:
df=df_origin.copy()

* 단계 1: 분석에 사용하지 않을 city, company_size, company_type 컬럼을 제거하시오.  

In [132]:
df = df.drop(["city", "company_size", "company_type"], axis = 1)

* 단계 2: 각 문자형 변수에 결측치(null/empty space)가 하나라도 존재하는 행은 모두 제거하시오.  

In [133]:
(df=='').sum()

enrollee_id               0
city_development_index    0
gender                    0
relevant_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
last_new_job              0
training_hours            0
target                    0
Xgrp                      0
dtype: int64

In [134]:
df = df.dropna()

* 단계 3: experience 변수 값이 ‘>20’ 또는 ‘<1’인 값을 제거하고 experience 컬럼의 타입을 정수형으로 변환하시오.  

In [135]:
df = df.loc[~(df["experience"].isin([">20", "<1"])), ]

In [136]:
df['experience']=df.loc[:,'experience'].astype('int')

* 단계 4: last_new_job 변수의 값이 ‘>4’ 또는 ‘never’인 값을 제거하고 해당변수의 타입을 정수형으로 변환하시오.  

In [137]:
df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]
df["last_new_job"] = df.loc[:,"last_new_job"].astype("int")

  
전처리 수행 이후 행 개수는 7,522이며 전처리가 완료된 데이터를 base 객체로 지정하고 이를 사용하여 문제를 풀이하시오.  

In [138]:
len(df)

7522

In [139]:
df_base = df.copy()

#### Q1. (base를 사용하여)관련 분야 경험 여부(relevant_experience)에 따른 전배 희망 여부(target)를 기술통계량으로 확인하고자 한다. 관련 분야 경험이 없는 수료자 중 전배를 희망하는 수료자의 비율을 A, 관련 분야 경험이 있는 수료자 중 전배를 희망하는 수료자의 비율을 B라 할 때, A/B를 구하시오.
※ 관련 경험이 없는 사람은 relevant_experience변수의 값이 ‘No relevant experience’인 사람으로 정의한다.  
※ 관련 경험이 있는 사람은 relevant_experience변수의 값이 ‘Has relevant experience’인 사람으로 정의한다.  
※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [140]:
df_base["relevant_experience"].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [141]:
df_base["target"].unique()

array([1., 0.])

In [142]:
df_base_no  = df_base.loc[df_base["relevant_experience"] == "No relevant experience"]
df_base_has = df_base.loc[df_base["relevant_experience"] == "Has relevant experience"]

In [143]:
df_base_no["target"].value_counts()

0.0    872
1.0    541
Name: target, dtype: int64

In [195]:
df_base_no["target"].value_counts(normalize = True)

0.0    0.617127
1.0    0.382873
Name: target, dtype: float64

In [144]:
stat_no = df_base_no["target"].value_counts(normalize = True)[1]
stat_no

0.38287331917905165

In [145]:
stat_has = df_base_has["target"].value_counts(normalize = True)[1]
stat_has

0.21591095105581928

In [146]:
round(stat_no / stat_has, 2)

1.77

#### Q2. (base를 사용하여) 전배 희망 여부(target)에 영향을 주는 변수들을 확인하고자 한다. 다음 절차에 따라 로지스틱 회귀분석을 수행하고 질문에 답하시오.
* 단계 1: gender, relevant_experience, enrolled_university, education_level, major_discipline 변수로부터 더미 변수들을 생성한다. 단, 각 변수로부터 더미 변수를 생성할 때 마지막으로 등장하는 범주는 제외하도록 한다. (여기서 마지막으로 등장하는 범주란, 각 컬럼의 값을 사전 순으로 나열하였을 때 마지막으로 등장하는 값이다. 예를 들어, ‘col’ 변수의 범주가 \[‘A’,’C’,’B’,’B’,’C’,’A’\]의 값을 가진다면 사전 상의 마지막 값인 C가 제외된다)
* 단계 2: 단계 1에서 생성한 더미 변수와 city_development_index, experience, last_new_job, training_hours, target, Xgrp 변수를 결합하여 새로운 데이터셋(데이터셋명: job2, 이 데이터셋은 문제 3에서도 활용)을 구성한다. 이 때, target, Xgrp를 제외한 데이터셋의 컬럼은 아래 순서에 따르도록 한다.  
\- city_development_index  
\- experience  
\- last_new_job  
\- training_hours  
\- gender의 더미 변수  
\- relevant_experience의 더미 변수  
\- enrolled_university의 더미 변수  
\- education_level의 더미 변수  
\- major_discipline의 더미 변수  

* 단계 3. 단계 2에서 구성한 데이터셋 job2로 다음 조건에 따라 상수항(Intercept)이 포함된 로지스틱 회귀분석을 수행한다.  
\- 종속 변수 : target  
\- 독립 변수(총 16개) : target과 Xgrp를 제외한 나머지 변수  
\- 회귀식에 포함되는 독립 변수의 순서를 컬럼의 순서와 일치시킨다.  

상수항을 제외한 나머지 변수들에 대한 Odds Ratio중 가장 큰 값을 기술하시오.  

$$ x_i 의\: Odds Ratio = \frac{odds(P(Y=1|x_1,\: \dots, \: x_i + 1,\: \dots ,\: x_n))}{odds(P(Y=1|x_1,\: \dots ,\:  x_i,\: \dots ,\: x_n)} $$

※ LogisticRegression() 클래스의 인자 C는 100000으로 지정하시오.  
※ LogisticRegression() 클래스의 인자 random_state는 123으로 지정하시오.  
※ 정답은 소수점 셋째 자리에서 버림하여 둘째 자리까지 출력하시오.  
(정답 예시: 0.123)

---

※ LogisticRegression() 클래스의 인자 C는 100000으로 지정하시오.  
※ LogisticRegression() 클래스의 인자 random_state는 123으로 지정하시오.  

# params

In [147]:
C = 100000
random_state = 123

* 단계 1: gender, relevant_experience, enrolled_university, education_level, major_discipline 변수로부터 더미 변수들을 생성한다. 단, 각 변수로부터 더미 변수를 생성할 때 마지막으로 등장하는 범주는 제외하도록 한다. (여기서 마지막으로 등장하는 범주란, 각 컬럼의 값을 사전 순으로 나열하였을 때 마지막으로 등장하는 값이다. 예를 들어, ‘col’ 변수의 범주가 \[‘A’,’C’,’B’,’B’,’C’,’A’\]의 값을 가진다면 사전 상의 마지막 값인 C가 제외된다)

In [148]:
df_t = pd.DataFrame(dict(aa = ["a", "A", "C", "D"]))
pd.get_dummies(df_t)

aa_A  aa_C  aa_D  aa_a
0     0     0     0     1
1     1     0     0     0
2     0     1     0     0
3     0     0     1     0

> aa_a를 삭제하란 말이네

In [149]:
df_cat = df_base[["gender", "relevant_experience", "enrolled_university", "education_level", "major_discipline"]]
df_cat_dum = pd.get_dummies(df_cat)

In [150]:
df_cat_dum

gender_Female  gender_Male  gender_Other  \
8                  0            1             0   
11                 0            1             0   
19                 1            0             0   
20                 0            1             0   
21                 0            1             0   
...              ...          ...           ...   
19149              0            1             0   
19150              1            0             0   
19152              1            0             0   
19153              0            1             0   
19154              0            1             0   

       relevant_experience_Has relevant experience  \
8                                                1   
11                                               1   
19                                               1   
20                                               1   
21                                               1   
...                                            ...   
19149                                            1   
19150                                            1   
19152                                            1   
19153                                            0   
19154                                            1   

       relevant_experience_No relevant experience  \
8                                               0   
11                                              0   
19                                              0   
20                                              0   
21                                              0   
...                                           ...   
19149                                           0   
19150                                           0   
19152                                           0   
19153                                           1   
19154                                           0   

       enrolled_university_Full time course  \
8                                         0   
11                                        0   
19                                        0   
20                                        0   
21                                        0   
...                                     ...   
19149                                     0   
19150                                     0   
19152                                     0   
19153                                     0   
19154                                     0   

       enrolled_university_Part time course  \
8                                         0   
11                                        0   
19                                        0   
20                                        0   
21                                        0   
...                                     ...   
19149                                     0   
19150                                     0   
19152                                     0   
19153                                     0   
19154                                     0   

       enrolled_university_no_enrollment  education_level_Graduate  \
8                                      1                         1   
11                                     1                         1   
19                                     1                         1   
20                                     1                         0   
21                                     1                         0   
...                                  ...                       ...   
19149                                  1                         0   
19150                                  1                         1   
19152                                  1                         1   
19153                                  1                         1   
19154                                  1                         1   

       education_level_Masters  education_level_Phd  major_discipline_Arts  \
8                            0                    0        

In [151]:
df_cat_dum = df_cat_dum.drop(["gender_Other", "relevant_experience_No relevant experience", 
                              "enrolled_university_no_enrollment", "education_level_Phd",
                              "major_discipline_STEM"],
                             axis = 1)
df_cat_dum.head(1)

gender_Female  gender_Male  relevant_experience_Has relevant experience  \
8              0            1                                            1   

   enrolled_university_Full time course  enrolled_university_Part time course  \
8                                     0                                     0   

   education_level_Graduate  education_level_Masters  major_discipline_Arts  \
8                         1                        0                      0   

   major_discipline_Business Degree  major_discipline_Humanities  \
8                                 0                            0   

   major_discipline_No Major  major_discipline_Other  
8                          0                       0

>  각 범주의 nunique 구해서 하나씩 저장해서 컬럼에서 빼도 되겠지만 저 정도는 빨리 풀어봅시다.

* 단계 2: 단계 1에서 생성한 더미 변수와 city_development_index, experience, last_new_job, training_hours, target, Xgrp 변수를 결합하여 새로운 데이터셋(데이터셋명: job2, 이 데이터셋은 문제 3에서도 활용)을 구성한다. 이 때, target, Xgrp를 제외한 데이터셋의 컬럼은 아래 순서에 따르도록 한다.  
\- city_development_index  
\- experience  
\- last_new_job  
\- training_hours  
\- gender의 더미 변수  
\- relevant_experience의 더미 변수  
\- enrolled_university의 더미 변수  
\- education_level의 더미 변수  
\- major_discipline의 더미 변수  

In [152]:
col_select = ["training_hours","experience","last_new_job","city_development_index", "target", "Xgrp"]
df_job2 = pd.concat([df_base[col_select],
                     df_cat_dum],
                    axis = 1)

* 단계 3. 단계 2에서 구성한 데이터셋 job2로 다음 조건에 따라 상수항(Intercept)이 포함된 로지스틱 회귀분석을 수행한다.  
\- 종속 변수 : target  
\- 독립 변수(총 16개) : target과 Xgrp를 제외한 나머지 변수  
\- 회귀식에 포함되는 독립 변수의 순서를 컬럼의 순서와 일치시킨다.  

상수항을 제외한 나머지 변수들에 대한 Odds Ratio중 가장 큰 값을 기술하시오.  

$$ x_i 의\: Odds Ratio = \frac{odds(P(Y=1|x_1,\: \dots, \: x_i + 1,\: \dots ,\: x_n))}{odds(P(Y=1|x_1,\: \dots ,\:  x_i,\: \dots ,\: x_n)} $$


※ 정답은 소수점 셋째 자리에서 버림하여 둘째 자리까지 출력하시오.  
(정답 예시: 0.123)

In [153]:
df_job2.shape

(7522, 18)

In [154]:
df_job2.head(1)

training_hours  experience  last_new_job  city_development_index  target  \
8            46.0           7             1                    0.92     1.0   

    Xgrp  gender_Female  gender_Male  \
8  train              0            1   

   relevant_experience_Has relevant experience  \
8                                            1   

   enrolled_university_Full time course  enrolled_university_Part time course  \
8                                     0                                     0   

   education_level_Graduate  education_level_Masters  major_discipline_Arts  \
8                         1                        0                      0   

   major_discipline_Business Degree  major_discipline_Humanities  \
8                                 0                            0   

   major_discipline_No Major  major_discipline_Other  
8                          0                       0

In [155]:
model_lr = LogisticRegression(C=C, random_state=random_state, max_iter=1000)

In [156]:
model_lr.fit(X = df_job2.drop(["target", "Xgrp"], axis = 1),
             y = df_job2["target"])

LogisticRegression(C=100000, max_iter=1000, random_state=123)

In [157]:
model_lr.coef_

array([[-9.32218671e-04, -2.85372337e-02,  9.51271677e-02,
        -6.14438514e+00, -1.94534782e-01, -1.62056957e-01,
        -7.68017011e-01,  5.15064359e-01, -2.82623097e-01,
         2.81264029e-01, -2.12842743e-02,  2.99999505e-01,
         1.23687744e-01,  2.38361108e-01,  4.04452723e-01,
        -4.42178080e-01]])

In [158]:
round(np.exp(model_lr.coef_).round(3).max(),3).astype(str)[:-1]

'1.67'

### Q3. (job2를 이용하여) 전체 데이터를 Train과 Test Set으로 나누고, Train Set으로 학습한 모델을 Test Set에 적용하여 모델을 평가하고자 한다. 다음 절차에 따라 분석을 수행하고 질문에 답하시오.

* 단계 1: 문제 5번 2단계에서 구성한 데이터셋 job2 에서 Xgrp 컬럼의 값이 ‘train’인 경우 Train Set으로,‘test’인 경우 Test Set으로 정의하여 분할한다.
* 단계 2: 아래 가이드에 따라 Train Set으로 K-NN 분류 모델을 학습하고, 이 모델을 Test Set에 적용한다.  
\- 종속 변수: 전배 희망 여부(target)  
\- 독립 변수(총 16개): 전배 희망 여부(target)와 Train/Test set 구분 변수(Xgrp)를 제외한 모든 변수  
\- Euclidean 거리 기준 가장 가까운 5개 데이터의 ‘전배 희망 여부(target)’를 활용하여 예측  
* 단계 3: 예측 결과를 바탕으로 아래 정의된 지표 A를 계산하여 기술하시오.

$$ 𝐴=((\#\: of\: true\: positive)+(\#\: of\: true\: negative))/((\#\: of\: total\: data)) $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

* 단계 1: 2단계에서 구성한 데이터셋 job2 에서 Xgrp 컬럼의 값이 ‘train’인 경우 Train Set으로,‘test’인 경우 Test Set으로 정의하여 분할한다.


In [166]:
df_job2["Xgrp"].unique()

array(['train', 'test'], dtype=object)

In [167]:
# df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
# df_test  = df_job2.loc[df_job2["Xgrp"] != "train", ]

In [168]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ].drop("Xgrp", axis = 1)
df_test  = df_job2.loc[df_job2["Xgrp"] == "test" , ].drop("Xgrp", axis = 1)

* 단계 2: 아래 가이드에 따라 Train Set으로 K-NN 분류 모델을 학습하고, 이 모델을 Test Set에 적용한다.  
\- 종속 변수: 전배 희망 여부(target)  
\- 독립 변수(총 16개): 전배 희망 여부(target)와 Train/Test set 구분 변수(Xgrp)를 제외한 모든 변수  
\- Euclidean 거리 기준 가장 가까운 5개 데이터의 ‘전배 희망 여부(target)’를 활용하여 예측  


In [170]:
model = KNeighborsClassifier(n_neighbors = 5,    p=2,
    metric='minkowski')
model.fit(X = df_train.drop("target", axis = 1),
          y = df_train["target"])
pred = model.predict(df_test.drop("target", axis = 1))

In [171]:
pred[:4]

array([0., 0., 0., 0.])

* 단계 3: 예측 결과를 바탕으로 아래 정의된 지표 A를 계산하여 기술하시오.

$$ 𝐴=((\#\: of\: true\: positive)+(\#\: of\: true\: negative))/((\#\: of\: total\: data)) $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [173]:
from sklearn.metrics import accuracy_score
round(accuracy_score(y_true = df_test["target"], 
                     y_pred = pred), 2)

0.71